<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 06b</h1>
<h2>RCTs in plain view, in OLS and otherwise</h2>

Probably the most publicly visible randomized controlled trials (RCTs) are clinical trials for vaccines. But especially in modern clinical trials, it can be difficult to understand the statistical methods behind the science. Let's unpack the statistics in two useful examples.

<h2>Learning objectives:</h2>


1. Outcome data from randomized controlled trials can often be 0's and 1's, when the outcome is a health condition and the study is a clinical trial, say of a <b>vaccine</b>

<p>

2. Ordinary least squares (OLS) estimation of $y = \alpha + \beta x + \epsilon$ can still tell you $\alpha$, the average outcome in the control group ($x = 0$); and the average difference in the outcome between control ($x = 0$) and treatment ($x = 1$), i.e. the treatment effect, $\beta$
    
<p>
   
3. There are many other analysis methods too. Statisticians might use [Fisher's exact test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test) or a [Chi-Square test]()
<p>
    
4. Modern clinical trials and their writeups are often much more complex than this. For ECON 140, you need to be able to use OLS and understand its output. Interested in learning more? Perhaps check out biostatistics!

In [ ]:
library(haven)
library(tidyverse)

<h2>The Salk polio vaccine trials of 1954</h2>

[Poliomyelitis](https://en.wikipedia.org/wiki/Polio) is an infectious viral disease that can result in muscle weakenss and ultimately paralysis. A shockingly long list of [polio survivors](https://en.wikipedia.org/wiki/List_of_polio_survivors) includes Senator [Mitch McConnell](https://en.wikipedia.org/wiki/Mitch_McConnell) (b. 1942) of Kentucky, currently the minority leader and in the news in Fall 2023 because of unrelated health complications that are raising questions about his ability to continue in the Senate. Despite the usual patterns in the new politics of vaccination in the U.S., McConnell has remained a strong supporter of vaccines due in part to his childhood experiences with polio before the widespread advent of the vaccine.

[Dawson (2004)](https://journals-sagepub-com.libproxy.berkeley.edu/doi/10.1191/1740774504cn010xx) provides a recent review of the vaccine trials, and [Brownlee (1955)](https://www-tandfonline-com.libproxy.berkeley.edu/doi/abs/10.1080/01621459.1955.10501286) wrote a useful summary for the <i>Journal of the American Statistical Association</i> of the densely written [original report itself](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1622939/). 

In particular, here is a helpful passage by Brownlee on page 1,010:
<img src="images/brownlee-jasa55-p1010.png" width="400" />

There were other dimensions of the Salk vaccine trial, but these numbers are drawn from the cleanest part, a very large randomized controlled trial in which 200,745 randomly selected children received the vaccine, and 201,229 other children received a placebo and were in the control group. Children and their families were not told which group they were in, and their physicians and the study directors also did not know, a process called "double blinding." It is intended to keep behavior and reactions by children, families, and doctors the same in both groups.

Here are the key measures:

In [ ]:
treatment_total = 200745
treatment_polio = 33

control_total   = 201229
control_polio   = 115

Here are the proportions or probabilities, their difference, and their ratio:

In [ ]:
treatment_prate = treatment_polio/treatment_total
control_prate   = control_polio/control_total

treatment_prate*100000
control_prate*100000
(treatment_prate - control_prate)*100000
treatment_prate/control_prate

Because the probabilities are small, their levels and differences are hard to read. In the treatment group, the probability of getting polio was 0.016 percent, or about 16 in 100,000. In the control group, the probability was 0.057 percent, or 57 in 100,000. Treatment appeared to reduce the probability of illness by 41 percentage points, or by about 28.8 percent.

Here are the <b>odds</b> and the <b>odds ratio</b>:

In [ ]:
treatment_odds    = treatment_polio/(treatment_total - treatment_polio)
control_odds      = control_polio  /(control_total   - control_polio) 

treatment_odds
control_odds
treatment_odds/control_odds

This report can be summarized in a [contingency table](https://en.wikipedia.org/wiki/Contingency_table), a commonly used method to report key results. Note that by aggregating these groups into cells that are 33 people or more in size, the researchers have <i>anonymized</i> their data.

Statisticians might use [Fisher's exact test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test) to assess the statistical significance of results in a contingency table. This test looks at the odds ratio.

In [ ]:
fisher.test( matrix(c(treatment_polio, treatment_total, 
                      control_polio, control_total), 
                    ncol = 2) )

With a $p$-value that small, we easily reject the null hypothesis that the true odds ratio is 1, i.e., that there is no difference in the odds of contracting polio between control and treatment groups; and instead, we fail to reject that there is a protective treatment effect of the vaccine.

Statisticians might also look at Chi-Square ($\chi^2$) test, also called a proportion test here in ``R``, along with an "X-squared" statistic reported.

Be careful about the order of the elements in the matrix, which here is transposed compared to what `fisher.test()` wants. Gotta love it! And here, the proportions or probabilities are reported, rather than the odds ratios. But the outcome is qualitatively the same: 

In [ ]:
result <- prop.test( matrix(c(treatment_polio, control_polio, 
                              treatment_total, control_total), ncol = 2) )
print(result)

<hr>

These results were [very big news at the time they were released](https://en.wikipedia.org/wiki/Announcement_of_polio_vaccine_success): April 12, 1955, on the tenth anniversary of the death of Franklin Delano Roosevelt (FDR). Although medical historians now appear to believe that FDR suffered from Guillain-Barré syndrome rather than polio, FDR was a public symbol of the disease and had established an early [treatment center](https://www.nationalww2museum.org/war/articles/franklin-d-roosevelt-little-white-house-warm-springs) for similar paralytic diseases. See [Goldman et al., 2003](https://journals-sagepub-com.libproxy.berkeley.edu/doi/10.1177/096777200301100412) for details.

<hr>

<h3>OLS and synthetic microdata</h3>
    
Ordinary least squares (OLS) can return similar insights, but we need microdata rather than a summary table to show it. Such data are not easy to find, because privacy concerns typically dissuade investigators from publicly releasing their datasets.

Instead, we can create a synthetic dataset to show how OLS reveals similar patterns. A big drawback is that we can't check for balance across treatment and control in the covariates. Above, the second paragraph quoted from [Brownlee (1955)](https://www-tandfonline-com.libproxy.berkeley.edu/doi/abs/10.1080/01621459.1955.10501286) mentions how the original study showed how age distributions "appear reasonably similar."

Here is code that generates a synthetic dataset of 0's and 1's for the outcome variable `polio` and for a treatment indicator variable `treatment`:

In [ ]:
zerovec1 = matrix(0, nrow = 200745-33, ncol = 1)
onevec1  = matrix(1, nrow = 33, ncol = 1)
treatment = append(zerovec1, onevec1)

onevec2 = matrix(1, nrow = 200745, ncol = 1)

treatment = cbind(treatment, onevec2)

In [ ]:
zerovec3 = matrix(0, nrow = 201229-115, ncol = 1)
onevec3  = matrix(1, nrow = 115, ncol = 1)
control = append(zerovec3, onevec3)

zerovec4 = matrix(0, nrow = 201229, ncol = 1)

control = cbind(control, zerovec4)

In [ ]:
polio = rbind(treatment, control)
#head(polio)
#tail(polio)

In [ ]:
# To run OLS using lm(), we need to convert to a data frame
poliodata <- as.data.frame(polio)

In [ ]:
# This names the two columns, so we can refer to them in the call to lm()
names(poliodata)[1] <- "polio"
names(poliodata)[2] <- "treat"
#head(poliodata)

Finally we are ready to run OLS using `lm()` and compare results to the Fisher and $\chi^2$ tests.

In [ ]:
reg_polio <- lm(polio ~ treat, data = poliodata)
summary(reg_polio)

As before, the numbers are very small here. Let's extract the coefficients using `coef()` and upscale them.

In [ ]:
reg_polio_coef <- coef(reg_polio)
reg_polio_coef*100000
(reg_polio_coef[1]+reg_polio_coef[2])*100000

Here now we see the familiar results from before, and we have a standard error and a $t$-statistic attached to the treatment variable `treat`. We can definitively reject the null hypothesis here too, and the $p$-value at far right is similar to what we saw in the Fisher and $\chi^2$ tests.

<hr>

As a final exercise, we could also break the data into separate dataframes for treatment and control, and then we could run `t.test()` on the average outcome (`polio`) in each:

In [ ]:
controldata <- filter(poliodata, treat == 0)
treatdata   <- filter(poliodata, treat == 1)
t.test(controldata$polio, treatdata$polio)

Samesies. All of these approaches give us similar results. Of them all, economists often like OLS the best, perhaps because it is a tool with which they are most familiar. As we will see in ECON 140, OLS is also useful in many circumstances, not just this one.


<h2>The Pfizer/BioNTech COVID-19 trials of 2020</h2>

[COVID-19 (SARS-CoV-2)](https://en.wikipedia.org/wiki/SARS-CoV-2) was detected in China in late 2019 and swept across the globe in 2020, resulting in a declaration by the World Health Organization (WHO) of a pandemic on March 11, 2020.

The U.S. government's [Operation Warp Speed](https://www.gao.gov/products/gao-21-319) supported the development of several vaccines to combat COVID-19. The writeup of the phase-3 clinical trial of the mRNA vaccine developed by Pfizer/BioNTech was released electronically on the website of the <i>New England Journal of Medicine</i> on December 10, 2020 ([Polack et al., 2020](https://www-nejm-org.libproxy.berkeley.edu/doi/full/10.1056/nejmoa2034577)). Later that month, the article revealing results of the phase-3 trial of the Moderna vaccine was similarly released at NEJM ([Baden et al., 2020](https://www-nejm-org.libproxy.berkeley.edu/doi/full/10.1056/nejmoa2035389)).

Microdata from these studies are not available as far as I can tell. But as in the case of the polio vaccine study, we know something that looks like a contingency table, found in Table 2 on page 2,612:

<img src="images/polack-etal-njem20-table2.png" width="800" />

There is a lot of information here, including a more scientific look at person-years of exposure. But a simple breakdown is as follows:

In [ ]:
# From Polack et al. (NEJM 2020) Table 2, top row
# Rate of Covid-19 occurrence in the treatment (BNT162b2) group 
pfizer_tr = 8/18198

# Rate of Covid-19 occurrence in the control (placebo) group 
pfizer_cr = 162/18325

pfizer_tr*100000
pfizer_cr*100000

In words, the simple cut of the data shows that the rate of COVID-19 was about 44 per 100,000 in the treatment group and about 884 per 100,000 in the control group.

Here is what ([Polack et al., 2020](https://www-nejm-org.libproxy.berkeley.edu/doi/full/10.1056/nejmoa2034577)) have to say about comparing these two, found on page 2,605:

<img src="images/polack-etal-nejm20-p2605.png" width="400" />

Here are the IRR and efficacy measures they are talking about:

In [ ]:
IRR_p = pfizer_tr/pfizer_cr
pfizer_efficacy = 100*(1 - IRR_p)
IRR_p
pfizer_efficacy

In words, the incidence rate ratio (IRR), the rate of confirmed COVID-19 in the treatment group to the rate in the control group, here was about 5 percent. The efficacy, calculated as shown, is thus about 95 percent.

How does the efficacy of the Pfizer/BioNTech vaccine against COVID-19 compare to the efficacy of the Salk polio vaccine?

In [ ]:
IRR_salk = treatment_prate/control_prate
salk_efficacy = 100*(1 - IRR_salk)
IRR_salk
salk_efficacy

There were 65 years of technological innovation separating the two trials, but the difference in efficacy is striking. Pfizer/BioNTech's vaccine was remarkable. (So was Moderna's.)

Finally, here is a redo of Fisher's exact test for the Pfizer-BioNTech contingency table:

In [ ]:
fisher.test( matrix(c(8, 18198, 162, 18325), ncol = 2) )

Mirroring what is stated in the NEJM article, a real home run is shown here in the Fisher test. A statistically significant reduction in COVID-19 prevalence with a $p$-value that is something like 22 quadrillionths if I get my scientific notation right. A vanishingly small chance that sampling error alone could explain the result. We firmly reject the null hypothesis that the Pfizer-BioNTech vaccine had no efficacy.

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>